
# Cambios Recientes en los Códigos Postales de México (2020–2023)

## Actualizaciones a Nivel Nacional por Año

1. **Cambios en 2020:** No se encontró un número oficial públicamente divulgado sobre cuántos códigos postales se modificaron en 2020. Sin embargo, los datos disponibles sugieren que fue un año de pocos cambios netos en comparación con años posteriores. Dado que 2022 fue destacado como un año **récord** en cambios (ver abajo), se infiere que 2020 tuvo **considerablemente menos** modificaciones (probablemente solo algunos cientos de nuevos códigos) a nivel nacional. *Esto coincide con la relativa desaceleración en desarrollos urbanos durante la etapa inicial de la pandemia de COVID-19, lo cual habría limitado la creación de nuevos códigos postales en 2020.* (No hay una cifra exacta publicada en las fuentes conectadas para este año.)

2. **Cambios en 2022:** Fue el año con **mayor número de modificaciones de códigos postales** en México en tiempos recientes. De acuerdo con un análisis de GeoPostcodes, en 2022 se **introdujeron o cambiaron 837 códigos postales** a nivel nacional. Este pico refleja el **dinámico crecimiento urbano** y ajustes administrativos en distintas regiones del país durante ese año. En otras palabras, **837 nuevas zonas** recibieron códigos postales (o se actualizaron códigos existentes) solo en 2022, marcando un récord en la década reciente.

3. **Cambios en 2023:** Tampoco existe una cifra oficial publicada para 2023, pero las tendencias indican que el número de modificaciones fue **menor que el de 2022** (posiblemente en el rango de algunos pocos cientos de nuevos códigos). GeoPostcodes destacó únicamente a 2022 como el año de mayor actualización, lo que implica que **2023 no superó ese nivel**. Para tener contexto, el número total de códigos postales en México aumentó de alrededor de **\~33 mil** a inicios de la década a **más de 36 mil** hacia 2025, crecimiento que incluye el gran salto de 2022. Es razonable asumir que 2023 contribuyó con cierta porción de ese incremento (aunque menor que 2022). Las evidencias de solicitudes de nuevos códigos en 2023 corroboran que siguieron asignándose códigos en zonas de desarrollo, pero sin alcanzar la magnitud excepcional del año previo.

## Enfoque en la Zona del Valle de México (CDMX y Edomex)

La zona metropolitana del Valle de México —que abarca la **Ciudad de México** (CDMX) y el **Estado de México** (Edomex)— concentra una porción importante de los códigos postales del país. Actualmente, la Ciudad de México cuenta con **1,102 códigos postales**, mientras que el Estado de México posee **2,354 códigos**. Esto suma más de 3,400 códigos postales solo en la región metropolitana, reflejando su alta densidad de colonias y localidades.

Dado el intenso desarrollo urbano en el Valle de México, **muchos de los cambios de códigos postales ocurridos en 2020–2023 se concentraron en esta zona**. Por ejemplo, en 2023 se documentó una modificación en la Alcaldía **Xochimilco (CDMX)**, donde el código postal **16730 fue cambiado por el 13093** para actualizar la división postal de esa área. Asimismo, en octubre de 2023 se asignó un **nuevo código postal** para la zona de *Lomas de Tecamachalco* en el Estado de México, a solicitud de autoridades federales, con el fin de reconocer un inmueble y sector específico dentro de ese fraccionamiento. Estos casos ilustran cómo las **colonias en expansión o recién regularizadas** en la metrópoli recibieron códigos postales nuevos o ajustados en los años recientes.

En resumen, **2022 fue el año con más cambios** (837 nuevos códigos a nivel nacional), mientras que **2020 y 2023 tuvieron menos modificaciones** cada uno. Gran parte de estos cambios —especialmente las asignaciones de nuevos códigos— ocurrieron en zonas de crecimiento urbano como la **Ciudad de México y el Estado de México**, asegurando que la infraestructura postal se mantenga al día con la expansión de la mancha urbana en el Valle de México. Las cifras reflejan la importancia de mantener actualizado el *Catálogo Nacional de Códigos Postales* para **servir adecuadamente a las áreas urbanas en constante evolución**, evitando confusiones en la logística de correos y paquetería.

**Fuentes:** GeoPostcodes (actualizaciones globales de códigos postales, 2024); Portal oficial *codigo-postal.co* (estadísticas de códigos por estado); Solicitudes de asignación de CP (Transparencia SEPOMEX, 2023).


In [290]:
# Recarga el módulo
import importlib
import numpy as np
import load_Data
import pandas as pd
importlib.reload(load_Data)
from load_Data import LoadData
from fuzzywuzzy import fuzz
import pickle
from typing import Dict, Tuple
import unicodedata
import re
import os
from datetime import datetime
import pyarrow
import fastparquet


In [291]:
loader = LoadData("../")

In [292]:
# Cargar datasets originales
original_datasets = loader.load_original_datasets()

# Cargar datasets clasificados donde los códigos postales son nulos
null_datasets = loader.load_classified_null_datasets()
equal_datasets = loader.load_classified_equal_datasets()
diff_datasets = loader.load_classified_different_datasets()

# Cargar datasets procesados
processed_datasets = loader.load_processed_datasets()

# Cargar GeoDataFrame con índice de marginación
marginacion_gdf = loader.load_marginacion_gdf()

# Usar como DataFrame regular (sin geometrías)
marginacion_df = loader.load_marginacion_gdf(as_dataframe=True)

# Cargar catálogo de códigos postales
catalogo_cp = loader.load_postal_codes_catalog()

In [293]:
def rename_dataframe_columns(dataframes_dict):
    
    for name, df in dataframes_dict.items():
        df_renamed = df.rename(columns={
            'SUS_CP': 'CP',
            'postal_code_api': 'CP_INDEX',
            'SUS_COL': 'COLONIA',
            'SUS_DEL': 'MUNICIPIO',
            'NOM_ENT': 'ENTIDAD',
        })
        dataframes_dict[name] = df_renamed
    
    return dataframes_dict

In [294]:
# Aplicar renombramiento a todos los diccionarios
original_datasets = rename_dataframe_columns(original_datasets)
null_datasets = rename_dataframe_columns(null_datasets)
equal_datasets = rename_dataframe_columns(equal_datasets)
diff_datasets = rename_dataframe_columns(diff_datasets)


In [295]:
for dataset_name, df in processed_datasets.items():
     df_renamed = df.rename(columns={
            'SUS_CP': 'CP',
            'postal_code_catalogo': 'CP_INDEX',
            'SUS_COL': 'COLONIA',
            'SUS_DEL': 'MUNICIPIO',
            'NOM_ENT': 'ENTIDAD',
        })
     processed_datasets[dataset_name] = df_renamed

In [296]:
catalogo_cp = catalogo_cp.rename(columns={
    'd_codigo': 'CP',
    'd_asenta': 'COLONIA',
    'D_mnpio': 'MUNICIPIO',
    'd_estado': 'ENTIDAD',
})
    

In [297]:
marginacion_gdf = marginacion_gdf.rename(columns={
    'NOM_ENT': 'ENTIDAD',
    'NOM_MUN': 'MUNICIPIO'
})

In [298]:
empty_cp_datasets = {}
nonempty_cp_datasets = {}

for name, df in processed_datasets.items():
    # Creamos una máscara para NaN o strings vacíos
    mask_empty = df['CP_INDEX'].isna() | (df['CP_INDEX'].astype(str).str.strip() == '')
    # DataFrames con CP_INDEX vacío
    empty_cp_datasets[name] = df[mask_empty].copy()
    # DataFrames con CP_INDEX no vacío
    nonempty_cp_datasets[name] = df[~mask_empty].copy()

In [299]:
concatenated_equal = {
    year: pd.concat([equal_datasets[f"df_coordenadas_sustentantes_{year}_iguales"],
                     nonempty_cp_datasets[f"df_coordenadas_sustentantes_{year}_completos"]],
                    ignore_index=True)
    for year in [ "2020", "2021", "2022", "2023" ]
}

concatenated_null = {
    year: pd.concat([null_datasets[f"df_coordenadas_sustentantes_{year}_nulos"],
                     empty_cp_datasets[f"df_coordenadas_sustentantes_{year}_completos"]],
                    ignore_index=True)
    for year in [ "2020", "2021", "2022", "2023" ]
}


In [300]:
# convierte en marginacion_gdf …
marginacion_gdf['CP'] = (
    marginacion_gdf['CP']
    .astype(int)
    .astype(str)
    .str.zfill(5)
)




In [301]:

def clean_and_pad(df):
    # — 1) CP: convertir a int → str
    df['CP'] = df['CP'].astype(int).astype(str)
    # — 2) CP_INDEX: 
    #     a) convertir a str
    #     b) quitar “.0” al final
    df['CP_INDEX'] = (
        df['CP_INDEX']
        .fillna('')                                  # → '' en vez de NaN
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)        # limpia ".0"
    )
    
    # — 3) aplicar padding de un '0' a la izquierda solo cuando la longitud es 4
    for col in ['CP','CP_INDEX']:
        mask4 = df[col].str.len() == 4
        df.loc[mask4, col] = '0' + df.loc[mask4, col]
    
    return df

# aplicarlo a todos tus dataframes
for key, df in original_datasets.items():
    original_datasets[key] = clean_and_pad(df)




In [302]:
marginacion_gdf

,GM_2020,COLONIA,CP,MUNICIPIO,ENTIDAD,geometry
0,Bajo,aguilera,02900,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.15443 19.47346, -99.15616 ..."
1,Bajo,aldana,02910,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.1461 19.47323, -99.1473 19..."
2,Muy bajo,ampliacion del gas,02970,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.15522 19.46844, -99.15634 ..."
3,Bajo,ampliacion petrolera,02470,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.1965 19.4852, -99.19667 19..."
4,Medio,ampliacion san pedro xalpa,02719,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.21601 19.48404, -99.21667 ..."
...,...,...,...,...,...,...
5955,Bajo,jajalpa,54980,tultepec,edomex,"MULTIPOLYGON (((-99.1241 19.6583, -99.12502 19..."
5956,Medio,la providencia,54980,tultepec,edomex,"MULTIPOLYGON (((-99.13675 19.64888, -99.13642 ..."
5957,Bajo,santiago teyahualco,54980,tultepec,edomex,"MULTIPOLYGON (((-99.11802 19.66253, -99.11865 ..."
5958,Medio,ex hda la mariscala,54949,tultitlan,edomex,"MULTIPOLYGON (((-99.13577 19.62349, -99.13583 ..."


In [303]:
for dataset_name, df in original_datasets.items():
    print(f"Dataset: {dataset_name}")
    print(df.head())

Dataset: df_coordenadas_sustentantes_2020
                          COLONIA     CP       MUNICIPIO ENTIDAD  lat_centro  \
0  ampliacion  general jose vicen  57710  nezahualcoyotl  edomex   19.384366   
1                  santa catarina  56030        chiautla  edomex   19.549580   
2               nueva san antonio  56605          chalco  edomex   19.277787   
3                        caltenco  54665       coyotepec  edomex   19.785197   
4                      hueypoxtla  55670      hueypoxtla  edomex   19.910035   

   lng_centro  lat_principal  lng_principal CP_INDEX  
0  -99.009547      19.383591     -99.010130    57710  
1  -98.880199      19.549059     -98.883151    56030  
2  -98.892980      19.278850     -98.892805    56605  
3  -99.199699      19.785052     -99.202366           
4  -99.076913      19.910934     -99.075907    55670  
Dataset: df_coordenadas_sustentantes_2021
                          COLONIA     CP       MUNICIPIO ENTIDAD  lat_centro  \
0               torres de

In [304]:
# Función para normalizar espacios en columnas de texto
def normalizar_espacios(df, columnas):
    """
    Normaliza espacios en las columnas especificadas:
    - Convierte múltiples espacios, tabulaciones y saltos de línea en un solo espacio
    - Elimina espacios al inicio y final del texto
    """
    for col in columnas:
        if col in df.columns:
            # Primero verificamos que sea tipo string/object
            if df[col].dtype == 'object':
                # Reemplazamos cualquier secuencia de espacios con un solo espacio
                df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
                # Eliminamos espacios al inicio y final
                df[col] = df[col].str.strip()
    return df



In [305]:
# Aplicar la función a marginacion_gdf
marginacion_gdf = normalizar_espacios(marginacion_gdf, ['MUNICIPIO', 'COLONIA'])

In [306]:
for key, df in original_datasets.items():
    original_datasets[key] = normalizar_espacios(df, ['MUNICIPIO', 'COLONIA'])

In [307]:
for key, df in original_datasets.items():
    # Añadir columna IM_2020 con valores nulos
    df['IM_2020'] = ""
    # Añadir columna HASH_CP con strings vacíos
    df['HASH_CP'] = ""

    # Añadir columna Distance con valores nulos
    df['Distance'] = ""

    # Actualizar el DataFrame en el diccionario
    original_datasets[key] = df

In [308]:
for dataset_name, df in original_datasets.items():
    print(f"Dataset: {dataset_name}")
    print(df.head(15))

Dataset: df_coordenadas_sustentantes_2020
                          COLONIA     CP              MUNICIPIO ENTIDAD  \
0   ampliacion general jose vicen  57710         nezahualcoyotl  edomex   
1                  santa catarina  56030               chiautla  edomex   
2               nueva san antonio  56605                 chalco  edomex   
3                        caltenco  54665              coyotepec  edomex   
4                      hueypoxtla  55670             hueypoxtla  edomex   
5               san pedro atzompa  55770                tecamac    cdmx   
6         santa ana tlachiahualpa  55994            temascalapa  edomex   
7            santa maria cozotlan  55810            teotihuacan    cdmx   
8   jardines de los claustros iii  54920              tultitlan  edomex   
9                        acatepec  43050                huautla    cdmx   
10                    santiaguito  56217                texcoco  edomex   
11                 la providencia  54783             teolo

In [309]:
def guardar_original_datasets(agregar_timestamp=True):
    """
    Guarda los DataFrames del diccionario original_datasets en formato pickle y parquet
    
    Args:
        agregar_timestamp (bool): Si True, agrega timestamp a los nombres de archivos
    """
    # Rutas base
    pickle_dir = "/home/cesar_r/Documentos/Proyectos/IberoSocialData/comipems-inequality-clustering-mapping/Match/VersionesDataFrames/pickle"
    parquet_dir = "/home/cesar_r/Documentos/Proyectos/IberoSocialData/comipems-inequality-clustering-mapping/Match/VersionesDataFrames/parquet"
    
    # Crear directorios si no existen
    os.makedirs(pickle_dir, exist_ok=True)
    os.makedirs(parquet_dir, exist_ok=True)
    
    # Generar timestamp si se solicita
    timestamp = ""
    if agregar_timestamp:
        timestamp = f"_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    print("=== Guardando original_datasets ===")
    print(f"Pickle: {pickle_dir}")
    print(f"Parquet: {parquet_dir}")
    print(f"Timestamp: {timestamp if timestamp else 'No'}")
    print("-" * 50)
    
    # Guardar cada DataFrame del diccionario original_datasets
    for dataset_name, df in original_datasets.items():
        try:
            # Guardar como pickle
            pickle_filepath = os.path.join(pickle_dir, f"{dataset_name}{timestamp}.pkl")
            with open(pickle_filepath, 'wb') as f:
                pickle.dump(df, f)
            print(f"✓ Pickle guardado: {dataset_name}{timestamp}.pkl")
            
            # Guardar como parquet
            parquet_filepath = os.path.join(parquet_dir, f"{dataset_name}{timestamp}.parquet")
            df.to_parquet(parquet_filepath, index=False)
            print(f"✓ Parquet guardado: {dataset_name}{timestamp}.parquet")
            
        except Exception as e:
            print(f"✗ Error guardando {dataset_name}: {str(e)}")
    
    print("-" * 50)
    print(f"Proceso completado. {len(original_datasets)} DataFrames guardados en ambos formatos.")
    
    # Guardar también el diccionario completo como pickle
    dict_pickle_path = os.path.join(pickle_dir, f"original_datasets_dict{timestamp}.pkl")
    with open(dict_pickle_path, 'wb') as f:
        pickle.dump(original_datasets, f)
    print(f"✓ Diccionario completo guardado: original_datasets_dict{timestamp}.pkl")
    
    return {
        'pickle_dir': pickle_dir,
        'parquet_dir': parquet_dir,
        'archivos_guardados': len(original_datasets)
    }

In [ ]:
def guardar_original_datasets_csv(agregar_timestamp=True):
    """
    Guarda los DataFrames del diccionario original_datasets en formato CSV
    
    Args:
        agregar_timestamp (bool): Si True, agrega timestamp a los nombres de archivos
    """
    # Ruta base
    csv_dir = "/home/cesar_r/Documentos/Proyectos/IberoSocialData/comipems-inequality-clustering-mapping/Match/VersionesDataFrames/csv"
    
    # Crear directorio si no existe
    os.makedirs(csv_dir, exist_ok=True)
    
    # Generar timestamp si se solicita
    timestamp = ""
    if agregar_timestamp:
        timestamp = f"_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    print("=== Guardando original_datasets en CSV ===")
    print(f"Directorio: {csv_dir}")
    print(f"Timestamp: {timestamp if timestamp else 'No'}")
    print("-" * 50)
    
    archivos_guardados = []
    
    # Guardar cada DataFrame del diccionario original_datasets
    for dataset_name, df in original_datasets.items():
        try:
            # Guardar como CSV
            csv_filepath = os.path.join(csv_dir, f"{dataset_name}{timestamp}.csv")
            df.to_csv(csv_filepath, index=False, encoding='utf-8')
            archivos_guardados.append(csv_filepath)
            print(f"✓ CSV guardado: {dataset_name}{timestamp}.csv")
            
        except Exception as e:
            print(f"✗ Error guardando {dataset_name}: {str(e)}")
    
    print("-" * 50)
    print(f"Proceso completado. {len(archivos_guardados)} DataFrames guardados en formato CSV.")
    
    return {
        'csv_dir': csv_dir,
        'archivos_guardados': len(archivos_guardados),
        'rutas_archivos': archivos_guardados
    }

# Ejecutar la función
resultado_csv = guardar_original_datasets_csv(agregar_timestamp=True)

In [310]:
#resultado = guardar_original_datasets(agregar_timestamp=True)

In [311]:
for year in ["2020", "2021", "2022", "2023"]:
    dataset_name = f"df_coordenadas_sustentantes_{year}"
    df = original_datasets[dataset_name].copy()
    
    # Convertir CP a entero para comparación
    df['CP_numeric'] = df['CP'].astype(int)
    
    # Definir máscaras
    mask_cdmx    = (df['CP_numeric'] >= 1000)  & (df['CP_numeric'] < 17000)
    mask_edomex  = (df['CP_numeric'] >= 50000) & (df['CP_numeric'] <= 57999)
    mask_foraneo = ~(mask_cdmx | mask_edomex)  # todo lo que no sea cdmx ni edomex
    
    # Asignar ENTIDAD según máscara
    df.loc[mask_cdmx,    'ENTIDAD'] = 'cdmx'
    df.loc[mask_edomex,  'ENTIDAD'] = 'edomex'
    df.loc[mask_foraneo, 'ENTIDAD'] = 'foraneo'
    
    # Limpiar columna auxiliar y guardar
    df = df.drop('CP_numeric', axis=1)
    original_datasets[dataset_name] = df
    
    # Informe rápido
    print(f"Año {year} - ENTIDAD actualizada:")
    print(df['ENTIDAD'].value_counts())
    print("-" * 30)


Año 2020 - ENTIDAD actualizada:
ENTIDAD
edomex     15897
cdmx        9070
foraneo      742
Name: count, dtype: int64
------------------------------
Año 2021 - ENTIDAD actualizada:
ENTIDAD
edomex     15404
cdmx        8910
foraneo      649
Name: count, dtype: int64
------------------------------
Año 2022 - ENTIDAD actualizada:
ENTIDAD
edomex     10955
cdmx        2508
foraneo      333
Name: count, dtype: int64
------------------------------
Año 2023 - ENTIDAD actualizada:
ENTIDAD
edomex     11606
cdmx        6963
foraneo      124
Name: count, dtype: int64
------------------------------


In [312]:
resultado2 = guardar_original_datasets(agregar_timestamp=True)

=== Guardando original_datasets ===
Pickle: /home/cesar_r/Documentos/Proyectos/IberoSocialData/comipems-inequality-clustering-mapping/Match/VersionesDataFrames/pickle
Parquet: /home/cesar_r/Documentos/Proyectos/IberoSocialData/comipems-inequality-clustering-mapping/Match/VersionesDataFrames/parquet
Timestamp: _20250710_123414
--------------------------------------------------
✓ Pickle guardado: df_coordenadas_sustentantes_2020_20250710_123414.pkl
✓ Parquet guardado: df_coordenadas_sustentantes_2020_20250710_123414.parquet
✓ Pickle guardado: df_coordenadas_sustentantes_2021_20250710_123414.pkl
✓ Parquet guardado: df_coordenadas_sustentantes_2021_20250710_123414.parquet
✓ Pickle guardado: df_coordenadas_sustentantes_2022_20250710_123414.pkl
✓ Parquet guardado: df_coordenadas_sustentantes_2022_20250710_123414.parquet
✓ Pickle guardado: df_coordenadas_sustentantes_2023_20250710_123414.pkl
✓ Parquet guardado: df_coordenadas_sustentantes_2023_20250710_123414.parquet
-------------------------

---

Aplicar teorica de conjuntos para corregir inconsistencias en los datos, y de igual forma, para disminuir la complejidad algoritmica de la solución.

---

In [313]:
arr_municipios_gdf = marginacion_gdf['MUNICIPIO'].unique().tolist()
arr_municipios_catalogo = catalogo_cp['MUNICIPIO'].unique().tolist()
arr_municipios_original_datasets2020 = original_datasets['df_coordenadas_sustentantes_2020']['MUNICIPIO'].unique().tolist()
arr_municipios_original_datasets2021 = original_datasets['df_coordenadas_sustentantes_2021']['MUNICIPIO'].unique().tolist()
arr_municipios_original_datasets2022 = original_datasets['df_coordenadas_sustentantes_2022']['MUNICIPIO'].unique().tolist()
arr_municipios_original_datasets2023 = original_datasets['df_coordenadas_sustentantes_2023']['MUNICIPIO'].unique().tolist()

In [314]:
comunes2020 = np.intersect1d(arr_municipios_gdf, arr_municipios_original_datasets2020)
comunes2021 = np.intersect1d(arr_municipios_gdf, arr_municipios_original_datasets2021)
comunes2022 = np.intersect1d(arr_municipios_gdf, arr_municipios_original_datasets2022)
comunes2023 = np.intersect1d(arr_municipios_gdf, arr_municipios_original_datasets2023)

elements_exclusivos2020 = np.setdiff1d(arr_municipios_original_datasets2020, comunes2020)
elements_exclusivos2021 = np.setdiff1d(arr_municipios_original_datasets2021, comunes2021)
elements_exclusivos2022 = np.setdiff1d(arr_municipios_original_datasets2022, comunes2022)
elements_exclusivos2023 = np.setdiff1d(arr_municipios_original_datasets2023, comunes2023)

comunes2020_prima = np.intersect1d(arr_municipios_catalogo, arr_municipios_original_datasets2020)
comunes2021_prima = np.intersect1d(arr_municipios_catalogo, arr_municipios_original_datasets2021)
comunes2022_prima = np.intersect1d(arr_municipios_catalogo, arr_municipios_original_datasets2022)
comunes2023_prima = np.intersect1d(arr_municipios_catalogo, arr_municipios_original_datasets2023)

elements_exclusivos2020_prima = np.setdiff1d(arr_municipios_original_datasets2020, comunes2020_prima)
elements_exclusivos2021_prima = np.setdiff1d(arr_municipios_original_datasets2021, comunes2021_prima)
elements_exclusivos2022_prima = np.setdiff1d(arr_municipios_original_datasets2022, comunes2022_prima)
elements_exclusivos2023_prima = np.setdiff1d(arr_municipios_original_datasets2023, comunes2023_prima)

len(elements_exclusivos2020_prima), len(elements_exclusivos2021_prima), len(elements_exclusivos2022_prima), len(elements_exclusivos2023_prima)

(279, 232, 127, 2)

In [315]:
# Crear conjuntos base (más eficiente que arrays para operaciones de conjuntos)
municipios_marginacion = set(marginacion_gdf['MUNICIPIO'].unique())
municipios_catalogo = set(catalogo_cp['MUNICIPIO'].unique())

# Crear la unión de ambos conjuntos de referencia
municipios_referencia = municipios_marginacion.union(municipios_catalogo)

# Función para obtener municipios problemáticos por año
def obtener_municipios_problematicos(year):
    """
    Retorna municipios del dataset del año especificado que NO están
    ni en marginacion_gdf ni en catalogo_cp
    """
    municipios_dataset = set(original_datasets[f'df_coordenadas_sustentantes_{year}']['MUNICIPIO'].unique())
    return municipios_dataset - municipios_referencia

# Obtener municipios problemáticos para cada año
municipios_problematicos = {
    year: obtener_municipios_problematicos(year) 
    for year in ['2020', '2021', '2022', '2023']
}

# Mostrar resultados
for year, problematicos in municipios_problematicos.items():
    print(f"Año {year}: {len(problematicos)} municipios problemáticos")
    if problematicos:
        print(f"  Ejemplos: {list(problematicos)[:10]}")  # Mostrar primeros 5
    print()

# Si quieres obtener la longitud de cada conjunto como en tu código original
longitudes = tuple(len(municipios_problematicos[year]) for year in ['2020', '2021', '2022', '2023'])
print(f"Longitudes: {longitudes}")

Año 2020: 278 municipios problemáticos
  Ejemplos: ['san cristobal de las casas', 'tijuana', 'morelia', 'tehuipango', 'tepoztlan', 'san andres cholula', 'santa maria del tule', 'san antonio de la cal', 'san nicolas buenos aires', 'chapulhuacan']

Año 2021: 231 municipios problemáticos
  Ejemplos: ['san cristobal de las casas', 'tijuana', 'silao', 'tepoztlan', 'san andres cholula', 'guadalupe victoria', 'coronango', 'tecoman', 'comitan de dominguez', 'san agustin de las juntas']

Año 2022: 126 municipios problemáticos
  Ejemplos: ['tijuana', 'morelia', 'santiago yucuyachi', 'san andres cholula', 'jerecuaro', 'cuautepec', 'santiago nuyoo', 'z de trinidad sanchez santos', 'cintalapa', 'pahuatlan']

Año 2023: 2 municipios problemáticos
  Ejemplos: ['chicoloapan de juarez', 'acambay']

Longitudes: (278, 231, 126, 2)
